In [22]:
import tobii_research as tr
from pprint import pprint
import time

tracker = tr.find_all_eyetrackers()[0]
print(f"Connected to: {tracker.device_name} ({tracker.address})")

Connected to: Tobii Pro Fusion (tobii-prp://TPFC1-010202444001)


In [23]:
data = None
timeout = time.time() + 15

def callback(signal):
    global data
    data = signal

tracker.subscribe_to(tr.EYETRACKER_GAZE_DATA, callback, as_dictionary=True)

# Wait for data
while data is None and time.time() < timeout:
    time.sleep(0.1)

tracker.unsubscribe_from(tr.EYETRACKER_GAZE_DATA, callback)

pprint(data)
print("Monotonic clock nanoseconds:", time.monotonic_ns())

{'device_time_stamp': 90541258,
 'left_gaze_origin_in_user_coordinate_system': (nan, nan, nan),
 'left_gaze_origin_validity': 0,
 'left_gaze_point_in_user_coordinate_system': (nan, nan, nan),
 'left_gaze_point_on_display_area': (nan, nan),
 'left_gaze_point_validity': 0,
 'left_pupil_diameter': nan,
 'left_pupil_validity': 0,
 'right_gaze_origin_in_user_coordinate_system': (nan, nan, nan),
 'right_gaze_origin_validity': 0,
 'right_gaze_point_in_user_coordinate_system': (nan, nan, nan),
 'right_gaze_point_on_display_area': (nan, nan),
 'right_gaze_point_validity': 0,
 'right_pupil_diameter': nan,
 'right_pupil_validity': 0,
 'system_time_stamp': 149656943907}
Monotonic clock nanoseconds: 149656971307470


In [112]:
import time
import logging
import tobii_research as tr


logger = logging.getLogger(__name__)

class TimeProbe:
    def __init__(self):
        before: int = tr.get_system_time_stamp()
        utc: int = time.time_ns()
        after: int = tr.get_system_time_stamp()

        system_timestamp_at_utc: float = (before + after) // 2
        self.latency: int = after - before
        self.offset = utc - (system_timestamp_at_utc * 1_000)

    def __lt__(self, other: "TimeProbe") -> bool:
        return self.latency < other.latency
    
    def __eq__(self, other: "TimeProbe") -> bool:
        return self.latency == other.latency

In [ ]:
import datetime

# 1. Get the current synchronized UTC in nanoseconds
sync_ns = (tr.get_system_time_stamp() * 1_000) + 

# 2. Convert to seconds (as a float) for fromtimestamp
# 1e9 = 1,000,000,000
readable_time = datetime.datetime.fromtimestamp(sync_ns / 1e9, tz=datetime.timezone.utc)

print(f"Current Sync Time: {readable_time.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]}")


Current Sync Time: 2026-02-06 17:44:01.287


In [ ]:
datetime.datetime.fromtimestamp((tr.get_system_time_stamp() * 1_000 + min(TimeProbe() for _ in range(10)).offset) // 1, tz=datetime.timezone.utc)

datetime.datetime(2026, 2, 6, 17, 49, 34, tzinfo=datetime.timezone.utc)

In [118]:
datetime.datetime.fromtimestamp((tr.get_system_time_stamp() * 1_000 + min(TimeProbe() for _ in range(10)).offset) // 1_000_000)

ValueError: year 58071 is out of range